#### Google Palm LLM & API key setup

In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the API key
api_key = os.getenv('GOOGLE_API_KEY')

from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key, temperature=0.2)

# Print the instance to verify
print(llm)

/Users/rohansaxena/Desktop/LLM Project/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GoogleGenerativeAI
Params: {'model': 'models/text-bison-001', 'temperature': 0.2, 'top_p': None, 'top_k': None, 'max_output_tokens': None, 'candidate_count': 1}


#### Connect with database and ask some basic questions

In [2]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [3]:
db_user = "root"
db_password = os.getenv('SQL_PASSWORD')
db_host = "localhost"
db_name = "ad_kpis"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=12)

print(db.table_info)


CREATE TABLE example_1 (
	week_of DATETIME NOT NULL, 
	ad_spend FLOAT NOT NULL, 
	impressions INTEGER NOT NULL, 
	`#_of_clicks` INTEGER NOT NULL, 
	optin_page_visitors INTEGER NOT NULL, 
	opt_ins INTEGER NOT NULL, 
	booked_calls INTEGER NOT NULL, 
	live_calls INTEGER NOT NULL, 
	closes INTEGER NOT NULL, 
	price FLOAT NOT NULL, 
	revenue FLOAT NOT NULL, 
	cash_collected FLOAT NOT NULL, 
	PRIMARY KEY (week_of)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
12 rows from example_1 table:
week_of	ad_spend	impressions	#_of_clicks	optin_page_visitors	opt_ins	booked_calls	live_calls	closes	price	revenue	cash_collected
2023-10-09 00:00:00	2505.48	61892	944	492	52	7	5	0	4800.0	0.0	0.0
2023-10-16 00:00:00	4047.02	80938	1193	790	77	25	15	3	4800.0	14400.0	14400.0
2023-10-23 00:00:00	6719.6	140626	1810	1061	82	27	17	4	4800.0	19200.0	19200.0
2023-10-30 00:00:00	8258.97	181646	2213	1322	133	25	14	2	6800.0	13600.0	13600.0
2023-11-06 00:00:00	8009.99	187199	2319	1303	134	42	30	0	

In [4]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How much ad spend was there on the week of october 9th 2023?")

/Users/rohansaxena/Desktop/LLM Project/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
I0000 00:00:1722103329.928198 25693766 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1722103329.939366 25693766 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported




> Entering new SQLDatabaseChain chain...
How much ad spend was there on the week of october 9th 2023?
SQLQuery:SELECT ad_spend FROM example_1 WHERE week_of = '2023-10-09 00:00:00'
SQLResult: [(2505.48,)]
Answer:2505.48
> Finished chain.


Above is the correct answer 👍🏼

In [5]:
qn2 = db_chain.run("How much ad spend was there in the month of October 2023?")

/Users/rohansaxena/Desktop/LLM Project/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How much ad spend was there in the month of October 2023?
SQLQuery:SELECT SUM(ad_spend) FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
SQLResult: [(21531.06982421875,)]
Answer:21531.06982421875
> Finished chain.


This is the correct answer 👍🏼

In [6]:
qn3 = db_chain.run("What was the cost per opt in in October 2023?")



> Entering new SQLDatabaseChain chain...
What was the cost per opt in in October 2023?
SQLQuery:SELECT ad_spend / opt_ins AS cost_per_opt_in FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
SQLResult: [(48.18230731670673,), (52.558701552353895,), (81.9463426543445,), (62.09751674107143,)]
Answer:52.558701552353895
> Finished chain.


It was close here, but not quite. It got us the cost per opt in for each week, but didn't calculate the total for the month. 

Above, it returned a wrong query which generated an error during query execution. It thinks discount
table would have start and end date which is normally true but in our table there is no start or end date column.
One thing we can do here is run the query directly.

In [7]:
sql_code = """
SELECT SUM(ad_spend) / SUM(opt_ins) AS cost_per_opt_in FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

SELECT SUM(ad_spend) / SUM(opt_ins) AS cost_per_opt_in FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
 
SQLQuery:SELECT SUM(ad_spend) / SUM(opt_ins) AS cost_per_opt_in FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
SQLResult: [(62.59031925644985,)]
Answer:62.59
> Finished chain.


This is the correct answer. We will use this for few shot learning

#### The Main Tasks I want this to be able to execute (which is what I am going to write the few shot learning sample queries for)

1. Calculate cost per opt in (on a week by week basis and monthly basis)
2. Calculate cost per booked call (on a week by week basis and monthly basis)
3. Calculate ROAS (Return On Ad Spend which is Revenue / Ad Spend). Again, also on a week by week basis and monthly basis

In [8]:
# Calculate cost per booked call

qns4 = db_chain.run("SELECT SUM(ad_spend) / SUM(booked_calls) AS cost_per_booked_call FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(ad_spend) / SUM(booked_calls) AS cost_per_booked_call FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
SQLQuery:SELECT SUM(ad_spend) / SUM(booked_calls) AS cost_per_booked_call FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
SQLResult: [(256.322259812128,)]
Answer:256.32
> Finished chain.


In [9]:
qns5 = db_chain.run("SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
SQLQuery:SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
SQLResult: [(2.1921808988287297,)]
Answer:2.1921808988287297
> Finished chain.


In [10]:
qns6 = db_chain.run("SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2024-06-01' AND '2024-06-30")



> Entering new SQLDatabaseChain chain...
SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2024-06-01' AND '2024-06-30
SQLQuery:SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2024-06-01' AND '2024-06-30'
SQLResult: [(4.801367181646181,)]
Answer:4.801367181646181
> Finished chain.


#### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [11]:
few_shots = [
    {'Question' : "What was the cost per opt in in October 2023?",
     'SQLQuery' : "SELECT SUM(ad_spend) / SUM(opt_ins) AS cost_per_opt_in FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns3},
    {'Question': "What was the cost per booked call in October 2023?",
     'SQLQuery':"SELECT SUM(ad_spend) / SUM(booked_calls) AS cost_per_booked_call FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns4},
    {'Question': "What was the ROAS in October 2023?",
     'SQLQuery' : "SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31",
     'SQLResult': "Result of the SQL query",
     'Answer': qns5} ,
     {'Question' : "What was the return on ad spend in June 2024?" ,
      'SQLQuery': "SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2024-06-01' AND '2024-06-30",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns6}
]

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [12]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

/Users/rohansaxena/Desktop/LLM Project/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
I0000 00:00:1722103413.156216 25693766 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [13]:
to_vectorize

["What was the cost per opt in in October 2023? SELECT SUM(ad_spend) / SUM(opt_ins) AS cost_per_opt_in FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31' Result of the SQL query 62.59",
 "What was the cost per booked call in October 2023? SELECT SUM(ad_spend) / SUM(booked_calls) AS cost_per_booked_call FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31' Result of the SQL query 256.32",
 "What was the ROAS in October 2023? SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31 Result of the SQL query 2.1921808988287297",
 "What was the return on ad spend in June 2024? SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2024-06-01' AND '2024-06-30 Result of the SQL query 4.801367181646181"]

In [14]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [15]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "What was the return on ad spend in April 2024"})

[{'Answer': '4.801367181646181',
  'Question': 'What was the return on ad spend in June 2024?',
  'SQLQuery': "SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2024-06-01' AND '2024-06-30",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '2.1921808988287297',
  'Question': 'What was the ROAS in October 2023?',
  'SQLQuery': "SELECT SUM(revenue) / SUM(ad_spend) AS ROAS FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31",
  'SQLResult': 'Result of the SQL query'}]

In [16]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [17]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### Setting up PromptTemplete using input variables

In [18]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [19]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [20]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [21]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [22]:
new_chain("What was the cost per opt in in October 2023?")



> Entering new SQLDatabaseChain chain...
What was the cost per opt in in October 2023?
SQLQuery:SELECT SUM(ad_spend) / SUM(opt_ins) AS cost_per_opt_in FROM example_1 WHERE week_of BETWEEN '2023-10-01' AND '2023-10-31'
SQLResult: [(62.59031925644985,)]
Answer:62.59
> Finished chain.


{'query': 'What was the cost per opt in in October 2023?', 'result': '62.59'}

Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [23]:
new_chain("What was the cost per opt in in June 2024?")



> Entering new SQLDatabaseChain chain...
What was the cost per opt in in June 2024?
SQLQuery:SELECT SUM(ad_spend) / SUM(opt_ins) AS cost_per_opt_in FROM example_1 WHERE week_of BETWEEN '2024-06-01' AND '2024-06-30'
SQLResult: [(43.788340796493905,)]
Answer:43.78
> Finished chain.


{'query': 'What was the cost per opt in in June 2024?', 'result': '43.78'}

This is correct!

In [24]:
new_chain("What was the cost per booked call in June 2024?")



> Entering new SQLDatabaseChain chain...
What was the cost per booked call in June 2024?
SQLQuery:SELECT SUM(ad_spend) / SUM(booked_calls) AS cost_per_booked_call FROM example_1 WHERE week_of BETWEEN '2024-06-01' AND '2024-06-30'
SQLResult: [(195.1436926800272,)]
Answer:195.14
> Finished chain.


{'query': 'What was the cost per booked call in June 2024?',
 'result': '195.14'}

This is also correct :)